## 🌟 Scenario: Content Moderation System

### The Problem

You've joined a growing tech community platform that has **50,000 users** but only **3 moderators**:
- **Sarah** manually reviews posts (6 hours/day, 200+ posts in queue)
- **Mike** tries to help users improve content (rarely has time)
- **Lisa** identifies harmful content (can't keep up)

**Current Issues:**
- Takes 5-10 minutes per post to check safety, tone, and grammar manually
- Users don't understand why content is rejected
- No time to enhance approved content

### Your Solution

Build an **AI-Powered Content Moderation System** that:

1. **Classifies** content type (social media post / article / comment)
2. **Analyzes** safety, tone, and grammar **in parallel**
3. **Scores** and decides: approve or reject
4. **Enhances** approved content automatically
5. **Provides feedback** to users

**Expected Impact:** Reduce moderation time from 5-10 minutes to 30 seconds per post!

### Example Test Cases

Your system should handle:

**✅ Good Content (needs enhancement):**
```
just finished reading an amzing book about AI ethics! 
its really make me think about how we build responsible systems.
```
→ Approve, fix grammar, enhance

**⚠️ Problematic Content:**
```
I hate this stupid product! Complete waste of money.
```
→ Flag for aggressive language, suggest constructive rephrasing

---

## 📋 Challenge Overview

### Your Mission

Build an **AI-Powered Content Moderation & Enhancement System** that:
1. Analyzes user-submitted content (text posts)
2. Moderates for safety and quality
3. Provides improvement suggestions
4. Enhances approved content

### Why This Challenge?

This challenge combines **multiple agentic patterns** in a realistic scenario:
- **Routing**: Classify content type (social media post, article, comment)
- **Evaluator-Optimizer**: Assess content quality and iterate improvements
- **Parallelization**: Analyze multiple aspects simultaneously (tone, safety, grammar)
- **Orchestrator-Worker**: Coordinate the full moderation pipeline
- **Prompt Chaining**: Transform raw content through moderation → enhancement → finalization

---

## 🎓 Part 1: Framework Selection & Justification

### Task 1.1: Choose Your Framework

**Instructions:**
1. Review the 4 frameworks you learned
2. Select ONE framework for this challenge
3. Write a justification (150-200 words) explaining:
   - Why you chose this framework
   - What strengths make it suitable for this challenge
   - What trade-offs you considered
   - How its features align with the challenge requirements

**Available Frameworks:**
- CrewAI: Role-based agents, sequential/hierarchical processes
- LangGraph: Graph-based state management, conditional routing
- LlamaIndex: Data-centric, built-in RAG capabilities
- smolagents: Lightweight, tool-focused, minimal dependencies

---

### ✍️ YOUR FRAMEWORK SELECTION

**Selected Framework:** [WRITE YOUR CHOICE HERE]

**Justification:**

[WRITE YOUR JUSTIFICATION HERE - 150-250 words]

---

## 🛠️ Part 2: Setup & Configuration

### Task 2.1: Install Dependencies

Install your chosen framework and configure your API keys.

In [34]:
# TODO: Install your chosen framework and dependencies
# Your code here:
!pip install -U langgraph langchain_openai langchain_community

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


### Task 2.2: Configure API Keys & Model

In [35]:
import os
from getpass import getpass
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

load_dotenv()

llm = ChatOpenAI(
    model="gpt-4.1-mini",
    temperature=0,
    base_url=os.environ["OPENAI_BASE_URL"]
)

print("Environment configured. Model 'gpt-4o' is ready for orchestration.")

Environment configured. Model 'gpt-4o' is ready for orchestration.


---

## 🏗️ Part 3: Implementation

Build your **Content Moderation & Enhancement System** by implementing the following components:

### System Architecture

```
User Content Input
      |
      v
┌─────────────────┐
│  Router Agent   │ ──> Classify: Social Media / Article / Comment
└────────┬────────┘
         |
         v
┌─────────────────────────┐
│ Parallel Analysis       │
│  - Safety Check Agent   │ ──> Detect harmful content
│  - Tone Analyzer Agent  │ ──> Assess sentiment/tone
│  - Grammar Checker      │ ──> Identify language issues
└────────┬────────────────┘
         |
         v
┌─────────────────────────┐
│ Evaluator Agent         │ ──> Aggregate findings, score content
└────────┬────────────────┘
         |
    ┌────┴─────┐
    v          v
  REJECT    APPROVE
            |
            v
    ┌───────────────┐
    │  Optimizer    │ ──> Suggest improvements
    │  Agent        │
    └───────┬───────┘
            |
            v
    ┌───────────────┐
    │  Enhancer     │ ──> Apply improvements
    │  Agent        │
    └───────┬───────┘
            |
            v
    Final Enhanced Content
```

---

### Task 3.1: Router Agent (Routing Pattern)

**Requirements:**
- Create a router that classifies content into: "social_media", "article", or "comment"
- Route should be based on length, structure, and style
- Return the classification decision

In [36]:
# TODO: Implement Router Agent
# This agent analyzes content and classifies it

# Your code here:
from typing import TypedDict, List, Optional
import json
from langchain_core.messages import HumanMessage


class ModerationState(TypedDict):
    content: str
    content_type: str
    safety_report: str
    tone_report: str
    grammar_report: str
    score: int
    decision: str 
    suggestions: str
    final_content: str


def router_agent(state: ModerationState):
    """Classifies content into social_media, article, or comment."""
    
    prompt = f"""
    Analyze the following user content and classify it into one of these three categories:
    1. 'social_media': Short, informal, often uses hashtags or casual language.
    2. 'article': Long-form, structured, professional, or educational.
    3. 'comment': Brief responses, usually reactive or conversational.
    
    Content: "{state['content']}"
    
    Return ONLY a JSON object with the key "classification".
    """
    
    response = llm.invoke([HumanMessage(content=prompt)])

    result = json.loads(response.content.replace("```json", "").replace("```", ""))
    
    print(f"--- ROUTER: Classified content as {result['classification']} ---")
    
    return {"content_type": result['classification']}

### Task 3.2: Parallel Analysis Agents (Parallelization Pattern)

**Requirements:**
- Implement 3 agents that run in parallel:
  1. **Safety Checker**: Detect toxic, harmful, or inappropriate content
  2. **Tone Analyzer**: Assess sentiment (positive/negative/neutral) and professionalism
  3. **Grammar Checker**: Identify spelling, grammar, and clarity issues
- Each agent should return a structured assessment
- Execute them concurrently for efficiency

In [37]:
# TODO: Implement Parallel Analysis Agents
# These agents analyze different aspects simultaneously

# Your code here:
def safety_checker(state: ModerationState):
    """Detects toxic or harmful content."""
    prompt = f"Analyze this {state['content_type']} for safety. Is it toxic, hateful, or harmful?\nContent: {state['content']}"
    response = llm.invoke(prompt)
    return {"safety_report": response.content}

def tone_analyzer(state: ModerationState):
    """Assesses sentiment and professionalism."""
    prompt = f"Analyze the tone of this {state['content_type']}. Is it positive, negative, or neutral? Is it professional?\nContent: {state['content']}"
    response = llm.invoke(prompt)
    return {"tone_report": response.content}

def grammar_checker(state: ModerationState):
    """Identifies spelling and grammar issues."""
    prompt = f"Identify grammar and spelling issues in this {state['content_type']}. Do not fix them yet, just list them.\nContent: {state['content']}"
    response = llm.invoke(prompt)
    return {"grammar_report": response.content}

### Task 3.3: Evaluator Agent (Evaluator-Optimizer Pattern - Part 1)

**Requirements:**
- Aggregate results from the 3 parallel agents
- Calculate an overall content quality score (0-100)
- Make a decision: APPROVE (score ≥ 70) or REJECT (score < 70)
- For approved content, provide specific improvement suggestions

In [38]:
# TODO: Implement Evaluator Agent
# This agent aggregates findings and makes decisions

# Your code here:

def evaluator_agent(state: ModerationState):
    """Aggregates findings and scores content quality."""
    
    prompt = f"""
    You are a Senior Content Moderator. Review these three reports and provide a final evaluation:
    
    1. Safety: {state['safety_report']}
    2. Tone: {state['tone_report']}
    3. Grammar: {state['grammar_report']}
    
    Original Content: "{state['content']}"
    
    Decision Criteria:
    - Score 0-100 based on quality and safety.
    - If Safety identifies harmful content, Score MUST be below 50.
    - APPROVE if score >= 70, otherwise REJECT.
    
    Return ONLY a JSON object with:
    "score": (int),
    "decision": "APPROVE" or "REJECT",
    "suggestions": "List specific improvements if approved, or reasons if rejected"
    """
    
    response = llm.invoke([HumanMessage(content=prompt)])
    result = json.loads(response.content.replace("```json", "").replace("```", ""))
    
    print(f"--- EVALUATOR: Score {result['score']} | Decision: {result['decision']} ---")
    
    return {
        "score": result['score'],
        "decision": result['decision'],
        "suggestions": result['suggestions']
    }

### Task 3.4: Optimizer & Enhancer Agents (Prompt Chaining + Evaluator-Optimizer)

**Requirements:**
- **Optimizer Agent**: Generate specific improvements based on evaluator feedback
- **Enhancer Agent**: Apply improvements to create an enhanced version
- Implement as a chain: Original Content → Optimizer → Enhancer → Final Content
- (Optional) Add a re-evaluation loop if initial enhancement score is still low

In [39]:
# TODO: Implement Optimizer and Enhancer Agents
# These agents improve content based on feedback

# Your code here:
def optimizer_agent(state: ModerationState):
    """Generates an optimization strategy based on evaluator feedback."""
    if state['decision'] == "REJECT":
        return {"suggestions": "N/A - Content Rejected"}
        
    prompt = f"""
    Based on this feedback: "{state['suggestions']}", 
    create a specific plan to enhance the following content while keeping its original meaning.
    Content: "{state['content']}"
    """
    response = llm.invoke(prompt)
    return {"suggestions": response.content}

def enhancer_agent(state: ModerationState):
    """Applies the optimization plan to create the final version."""
    if state['decision'] == "REJECT":
        return {"final_content": state['content']}
        
    prompt = f"""
    Rewrite the following content based on this improvement plan: "{state['suggestions']}"
    Original Content: "{state['content']}"
    
    Provide ONLY the final enhanced text.
    """
    response = llm.invoke(prompt)
    return {"final_content": response.content}

### Task 3.5: Orchestrator (Orchestrator-Worker Pattern)

**Requirements:**
- Create a master orchestrator that coordinates the entire pipeline:
  1. Route content type
  2. Run parallel analysis
  3. Evaluate and decide
  4. If approved, optimize and enhance
  5. Return final result with metadata
- Handle both approval and rejection cases
- Provide clear logging of each step

In [40]:
# TODO: Implement Orchestrator
# This coordinates the entire moderation pipeline

# Your code here:
from langgraph.graph import StateGraph, END

workflow = StateGraph(ModerationState)

workflow.add_node("classify", router_agent)
workflow.add_node("safety", safety_checker)
workflow.add_node("tone", tone_analyzer)
workflow.add_node("grammar", grammar_checker)
workflow.add_node("evaluate", evaluator_agent)
workflow.add_node("optimize", optimizer_agent)
workflow.add_node("enhance", enhancer_agent)

workflow.set_entry_point("classify")

workflow.add_edge("classify", "safety")
workflow.add_edge("classify", "tone")
workflow.add_edge("classify", "grammar")

workflow.add_edge("safety", "evaluate")
workflow.add_edge("tone", "evaluate")
workflow.add_edge("grammar", "evaluate")

def routing_logic(state: ModerationState):
    return "optimize" if state["decision"] == "APPROVE" else END

workflow.add_conditional_edges("evaluate", routing_logic)

workflow.add_edge("optimize", "enhance")
workflow.add_edge("enhance", END)

app = workflow.compile()

---

## 🧪 Part 4: Testing

### Task 4.1: Test with Sample Content

Test your system with the provided examples representing different scenarios.

In [41]:
# Test Case 1: Clean social media post (should be approved and enhanced)
test_content_1 = """
just finished reading an amzing book about AI ethics! 
its really make me think about how we build responsible systems. 
highly recomend it to anyone in tech!
"""

# Test Case 2: Professional article excerpt (should be approved, might need minor fixes)
test_content_2 = """
Machine learning algorithms have transformed the healthcare industry over the past decade.
These systems now assist in diagnosis, treatment planning, and patient monitoring.
However, concerns about data privacy and algorithmic bias remain significant challenges
that researchers and practitioners must address to ensure equitable healthcare delivery.
"""

# Test Case 3: Short comment with grammar issues (should be approved but needs enhancement)
test_content_3 = "this is grate! i totally agree with ur point about ai safety its so important"

# Test Case 4: Content with potential safety issues (might be rejected or flagged)
test_content_4 = """
I hate this stupid product! Complete waste of money. 
The company is terrible and everyone should avoid them.
"""

test_cases = [
    ("Social Media Post with Errors", test_content_1),
    ("Professional Article", test_content_2),
    ("Short Comment", test_content_3),
    ("Potentially Problematic Content", test_content_4)
]

for title, content in test_cases:
    print(f"\n{'='*70}")
    print(f"RUNNING: {title}")
    print(f"{'='*70}")
    
    # Initialize state for the orchestrator
    inputs = {"content": content.strip()}
    
    # Run the graph
    final_state = app.invoke(inputs)
    
    # Display the results
    print(f"1. CLASSIFICATION: {final_state.get('content_type', 'N/A').upper()}")
    print(f"\n2. ANALYSIS RESULTS:")
    print(f"   - Safety:  {final_state.get('safety_report', 'N/A')[:120]}...")
    print(f"   - Tone:    {final_state.get('tone_report', 'N/A')[:120]}...")
    print(f"   - Grammar: {final_state.get('grammar_report', 'N/A')[:120]}...")
    
    print(f"\n3. EVALUATION:")
    print(f"   - Score:    {final_state.get('score', 0)}/100")
    print(f"   - Decision: {final_state.get('decision', 'N/A')}")
    
    if final_state.get('decision') == "APPROVE":
        print(f"\n4. ENHANCED VERSION:")
        print(f"   {final_state.get('final_content')}")
    else:
        print(f"\n4. FEEDBACK:")
        print(f"   {final_state.get('suggestions')}")


RUNNING: Social Media Post with Errors
--- ROUTER: Classified content as social_media ---
--- EVALUATOR: Score 85 | Decision: APPROVE ---
1. CLASSIFICATION: SOCIAL_MEDIA

2. ANALYSIS RESULTS:
   - Safety:  The content you provided is positive and constructive. It expresses enthusiasm about a book on AI ethics and encourages ...
   - Tone:    The tone of the social media post is positive. The user expresses enthusiasm and appreciation for the book about AI ethi...
   - Grammar: 1. "amzing" should be "amazing" (spelling error)  
2. "its" should be "it's" (missing apostrophe for contraction)  
3. "...

3. EVALUATION:
   - Score:    85/100
   - Decision: APPROVE

4. ENHANCED VERSION:
   Just finished reading an amazing book about AI ethics!  
It really makes me think about how we build responsible systems.  
Highly recommend it to anyone in tech!

RUNNING: Professional Article
--- ROUTER: Classified content as article ---
--- EVALUATOR: Score 90 | Decision: APPROVE ---
1. CLASSIFICATION: 

---

## 📊 Part 5: Reflection & Analysis

### Task 5.1: Pattern Usage Documentation

Document how you used each agentic pattern in your implementation.

### ✍️ YOUR PATTERN USAGE ANALYSIS

**1. Routing Pattern:**
- Where used: [this was used at the very beginning with the router_agent]
- Why effective: [figure out if a text is a short comment or a long article first, the other agents know whether to be super strict or more relaxed about the tone and style.]

**2. Parallelization Pattern:**
- Where used: [safety_checker, tone_analyzer, and grammar_checker]
- Why effective: [its much faster than doing one check after another]
- Performance benefit: [it cuts down the waiting time for the user]

**3. Evaluator-Optimizer Pattern:**
- Where used: [this was the "quality control" step where the Evaluator looked at all the reports and decided if the post was good enough to publish]
- How feedback loop works: [evaluator gives notes on whats wrong. If the post passes, the optimizer uses those notes to plan out exactly how to fix the issues]

**4. Prompt Chaining Pattern:**
- Where used: [for the "Approved" posts]
- Stages in chain: [1 Read the feedback. 2 Plan the improvements. 3 Write the corrected version]

**5. Orchestrator-Worker Pattern:**
- How orchestration is managed: [LangGraph acted as manager of the whole system.]
- Worker coordination: [manager makes sure safety, tone, grammar do their jobs first, then hands their notes to the evaluator to make the final call]

---

### Task 5.2: Challenges & Solutions

Reflect on difficulties you encountered and how you solved them.

### ✍️ YOUR CHALLENGES & SOLUTIONS

**Challenge 1:**
- Problem: [multiple "Invalid API Key" error]
- Solution: [cleared variables, added .env and used getpass to re enter key]

**Challenge 2:**
- Problem: [it was tricky to make sure the safety, tone, and grammar all finished their work before the Evaluator started.]
- Solution: [used LangGraph’s state management to "join" the branches]

**Challenge 3:**
- Problem: [DESCRIBE]
- Solution: [EXPLAIN]

---

### Task 5.3: Framework Reflection

Now that you've completed the challenge, reflect on your framework choice.

### ✍️ YOUR FRAMEWORK REFLECTION

**What worked well with your chosen framework?**

[LangGraph made it really easy to see how the data moves from one step to the next.]

**What was difficult or limiting?**

[The state was the hardest part to get right. In LangGraph, you have to define exactly what data is being passed around in a dictionary. If one agent (node) forgets to return a value or uses the wrong key name, the whole system crashes]

**Would you choose the same framework again? Why or why not?**

[yes,  It is good way for building agents. It was a little hard at first because no experience but once you get the hang of it, it’s actually super chill. ]

**What would you do differently next time?**

[YOUR RESPONSE HERE]

---

## 🎁 Bonus Challenges (Optional)

If you want to go further, try these enhancements:

### Bonus 1: Multi-Language Support
- Add a language detection agent
- Support content in at least 3 languages

### Bonus 2: Customizable Moderation Rules
- Allow users to set content policy preferences
- Adjust safety thresholds based on use case (e.g., strict for children's content)

### Bonus 3: Performance Optimization
- Measure execution time for each component
- Implement caching for repeated content
- Optimize parallel execution

### Bonus 4: Explainability Dashboard
- Create a visualization showing:
  - Agent decision flow
  - Confidence scores at each stage
  - Before/after content comparison

### Bonus 5: Iterative Re-evaluation
- If enhanced content scores < 90, run another optimization loop
- Limit to maximum 3 iterations to prevent infinite loops

---

In [42]:
# Optional: Implement your bonus challenges here

---

## 📝 Evaluation Criteria

Your implementation will be assessed on:

### Functionality (20 points)
- ✅ Router correctly classifies content types
- ✅ Parallel agents execute concurrently
- ✅ Evaluator makes appropriate approve/reject decisions
- ✅ Enhancement chain improves content quality
- ✅ Orchestrator coordinates full pipeline

### Pattern Implementation (20 points)
- ✅ Routing pattern clearly implemented
- ✅ Parallelization working correctly
- ✅ Evaluator-optimizer feedback loop functional
- ✅ Prompt chaining evident in enhancement
- ✅ Orchestrator-worker hierarchy clear

### Code Quality (20 points)
- ✅ Clean, readable code
- ✅ Proper error handling
- ✅ Good documentation/comments
- ✅ Framework best practices followed

### Reflection & Analysis ( **40 points** )
- ✅ Thoughtful framework justification
- ✅ Clear pattern usage documentation
- ✅ Honest challenge/solution discussion
- ✅ Insightful framework reflection

### Bonus Points (up to 10 extra points)
- Optional challenges attempted and completed

---

## 🎉 Conclusion

Congratulations on completing this challenge! You've built a sophisticated multi-agent system that combines multiple agentic patterns in a real-world scenario.

### Key Takeaways

Through this challenge, you've learned:
- How to select appropriate frameworks for specific tasks
- How to combine multiple agentic patterns effectively
- How to design complex multi-agent systems
- How to handle real-world challenges in agent development
- How to evaluate and reflect on your architectural decisions

### Next Steps

1. **Experiment**: Try implementing this challenge with a different framework
2. **Extend**: Add more sophisticated features (RAG, custom tools, memory)
3. **Deploy**: Consider how you'd productionize this system
4. **Share**: Document your learnings and share with the community

Keep building, keep learning, and keep pushing the boundaries of what's possible with agentic systems! 🚀

---

**Happy Coding!** 💻✨